In [1]:
from PIL import Image
from os import listdir
from keras.models import load_model
import numpy as np
from numpy import asarray
from numpy import expand_dims
from keras_facenet import FaceNet
import pickle
import cv2
import os
from mtcnn import MTCNN

In [2]:
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
model_face = FaceNet()
mtcnn = MTCNN()

In [3]:
myfile = open("data_processed.pkl", "rb")
database = pickle.load(myfile)
myfile.close()

In [4]:
# same procedure as training 
folder = 'attendance_class/'
file_list = os.listdir(folder)
predicted=[]

for filename in file_list:
    path = os.path.join(folder, filename)
    gbr1 = cv2.imread(folder + filename)

    harr = HaarCascade.detectMultiScale(gbr1,1.1,4)
    faces_mtcnn = mtcnn.detect_faces(gbr1)
    if len(harr)>0:
        x1, y1, width, height = harr[0] 
    elif len(faces_mtcnn) > 0:
        x1, y1, width, height = faces_mtcnn[0]['box']
    else:
        x1, y1, width, height = 1, 1, 10, 10
    
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height

    gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)  # Convert from OpenCV to PIL
    gbr_array = asarray(gbr)

    face = gbr_array[y1:y2, x1:x2]

    face = Image.fromarray(face)
    face = face.resize((160, 160))
    face = asarray(face)
    
    face = expand_dims(face, axis=0)
    signature = model_face.embeddings(face)
    
    min_dist=100
    identity=' '
    for key, value in database.items() :
        dist = np.linalg.norm(value-signature)
        if dist < min_dist:
            min_dist = dist
            identity = key
    print(identity)
    cv2.namedWindow("output", cv2.WINDOW_NORMAL)       
    cv2.resizeWindow("output", gbr1.shape[0],gbr1.shape[1]) 
    cv2.putText(gbr1,identity, (100,100),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
    cv2.rectangle(gbr1,(x1,y1),(x2,y2), (0,255,0), 2)
    cv2.imshow("output",gbr1)
    cv2.waitKey(0)
    predicted.append(identity)
        
cv2.destroyAllWindows()
print(predicted)

1/1 [==============================] - 3s 3s/step
unknown
1/1 [==============================] - 0s 81ms/step
pooja
1/1 [==============================] - 0s 74ms/step
ishan
1/1 [==============================] - 0s 80ms/step
aakash
1/1 [==============================] - 0s 73ms/step
abhinav
1/1 [==============================] - 0s 86ms/step
bhumit
1/1 [==============================] - 0s 81ms/step
kundan
1/1 [==============================] - 0s 77ms/step
aayan
1/1 [==============================] - 0s 77ms/step
karanpal
1/1 [==============================] - 0s 77ms/step
sajal
1/1 [==============================] - 0s 70ms/step
sukrit
1/1 [==============================] - 0s 77ms/step
ram_koti
1/1 [==============================] - 0s 76ms/step
vaibhav
1/1 [==============================] - 0s 68ms/step
vedant
1/1 [==============================] - 0s 75ms/step
chavi
1/1 [==============================] - 0s 76ms/step
lakshay
1/1 [==============================] - 0s 70ms/step
unk

In [5]:
# store the name of people present in a text file 
attendance = [name for name in predicted if name != 'unknown']

# enter the date for which you want to mark attendance
user_date = input("Enter the date (e.g., DD-MM_YYYY): ")

file_name = f"{user_date}.txt"

with open(file_name, 'w') as file:
    for item in attendance:
        file.write(str(item) + '\n')

print(f"Data has been stored in {file_name}")

Data has been stored in 22-10-2023.txt
